In [1]:
from collections import Counter, defaultdict
import re
import random
from copy import copy

import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk import everygrams
from tqdm import tqdm

In [2]:
import zipfile
from IPython.display import clear_output
with zipfile.ZipFile('corpora.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

clear_output()

**Реализуйте базовый частотный метод по Шерлоку Холмсу:**

подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);

возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;

расшифруйте их таким частотным методом.

In [3]:
tokenizer = RegexpTokenizer(r'\w+')
alphabet = ' абвгдежзийклмнопрстуфхцчшщъыьэюя'

def tokenize(text, tokenizer=tokenizer):
    text = ''.join([c for c in text if c.lower() in alphabet])
    return ' '.join(tokenizer.tokenize(text.lower()))


def get_freqs(text, min_freq=0, n_gram=1):
    freqs = dict()
    if n_gram > 1:
        text = [''.join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        if value/len(text) > min_freq:
            freqs[key] = value/len(text)
    return freqs


def generate_mapping(freqs):
    original = list(freqs.keys())
    replacements = np.random.choice(original, replace=False, size=len(freqs))
    mapping = dict()
    for original_char, replacement_char in zip(original, replacements):
        mapping[original_char] = replacement_char
    return mapping


def apply_mapping(text, mapping):
    return ''.join([mapping.get(c, 'ь') for c in text])


def get_reverse_mapping(corpus_freqs, text_freqs):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    reverse_mapping = dict()
    for text_char, text_freq in text_freqs_sorted:
        min_diff = 1
        best_char = None
        for corpus_char, corpus_freq in corpus_freqs_sorted:
            diff = abs(corpus_freq - text_freq)
            if diff < min_diff:
                best_char = corpus_char
                min_diff = diff

        reverse_mapping[text_char] = best_char
        corpus_freqs_sorted = [(char, freq) for char, freq in corpus_freqs_sorted if char !=best_char]
        
    return reverse_mapping


def char_accuracy(text1, text2):
    assert len(text1) == len(text2)
    right_chars = 0
    for c1, c2 in zip(text1, text2):
        right_chars += int(c1 == c2)
    return right_chars / len(text1)

In [5]:
np.random.seed(42)

with open('AnnaKarenina.txt' ,'r') as file:
    corpus1 = file.readlines()
    
with open('WarAndPeace.txt' ,'r') as file:
    corpus2 = file.readlines()
    
corpus = corpus1 + corpus2

tokenized_corpus = tokenize(' '.join(corpus))
corpus_freqs = get_freqs(tokenized_corpus)
mapping = generate_mapping(corpus_freqs)

In [7]:
with open('test.txt' ,'r') as file:
    text = file.readlines()

tokenized_text = tokenize(' '.join(text))
encoded_text = apply_mapping(tokenized_text, mapping)
text_freqs = get_freqs(encoded_text)
tokenized_text

'зовут его николаем петровичем кирсановым у него в пятнадцати верстах от постоялого дворика хорошее имение в двести душ или как он выражается с тех пор как размежевался с крестьянами и завел ферму в две тысячи десятин земли отец его боевой генерал года полуграмотный грубый но не злой русский человек всю жизнь свою тянул лямку командовал сперва бригадой потом дивизией и постоянно жил в провинции где в силу своего чина играл довольно значительную роль николай петрович родился на юге россии подобно старшему своему брату павлу о котором речь впереди и воспитывался до четырнадцатилетнего возраста дома окруженный дешевыми гувернерами развязными но подобострастными адъютантами и прочими полковыми и штабными личностями родительница его из фамилии колязиных в девицах а в генеральшах агафоклея кузьминишна кирсанова принадлежала к числу матушеккомандирш носила пышные чепцы и шумные шелковые платья в церкви подходила первая ко кресту говорила громко и много допускала детей утром к ручке на ночь их

In [8]:
encoded_text

'ашпимузюшувчьшнщзлугзмдшпчкзлуьчдрщвшпелуиувзюшупугэмвщыъщмчупздрмщхушмугшрмшэншюшуыпшдчьщухшдшфззучлзвчзупуыпзрмчуыифучнчуьщьушвупедщжщзмрэурумзхугшдуьщьудщалзжзпщнрэуруьдзрм эвщлчучуащпзнуйздлиупуыпзумерэкчуызрэмчвуазлнчушмзъузюшусшзпшбуюзвздщнуюшыщугшниюдщлшмвебуюдисебувшувзуаншбудиррьчбукзншпзьупроужчав урпшоумэвинунэльиуьшлщвышпщнургздпщусдчющышбугшмшлуычпчачзбучугшрмшэввшужчнупугдшпчвъччуюызупурчниурпшзюшукчвщучюдщнуышпшн вшуавщкчмзн виоудшн увчьшнщбугзмдшпчкудшычнрэувщуоюзудшррччугшышсвшурмщдфзлиурпшзлиусдщмиугщпниушуьшмшдшлудзк упгздзычучупшргчмепщнрэуышукзмедвщыъщмчнзмвзюшупшадщрмщуышлщушьдижзввебуызфзпелчуюипздвздщлчудщапэавелчувшугшышсшрмдщрмвелчущыцомщвмщлчучугдшкчлчугшньшпелчучуфмщсвелчунчквшрмэлчудшычмзн вчъщузюшучауйщлчнччуьшнэачвехупуызпчъщхущупуюзвздщн фщхущющйшьнзэуьиа лчвчфвщуьчдрщвшпщугдчвщынзжщнщуьукчрниулщмифзььшлщвычдфувшрчнщугефвезукзгъеучуфилвезуфзньшпезугнщм эупуъздьпчугшыхшычнщугздпщэуьшуьдзрмиуюшпшдчнщуюдшльшучулвшюшуышгирьщнщуызмзбуимдшлуьудикьзувщувшк учх

In [9]:
reverse_mapping = get_reverse_mapping(corpus_freqs, text_freqs)
decoded_text = apply_mapping(encoded_text, reverse_mapping)
decoded_text

'зоспр еьо тадолнем яервосайем давкнтосгм п теьо с ябртнуэнра севкрнц ор яокроблоьо усовадн цовохее аметае с усекра упх ала днд от сгвнюнеркб к рец яов днд внзмеюеснлкб к двекрчбтнма а знсел февмп с усе ргкбйа уекбрат земла ореэ еьо шоесож ьетевнл ьоун яолпьвнмортгж ьвпшгж то те злож впккдаж йелосед скщ юазтч ксощ рбтпл лбмдп домнтуоснл кяевсн шваьнуож яором уасазаеж а яокробтто юал с явосатэаа ьуе с калп ксоеьо йатн аьвнл уосолчто зтнйарелчтпщ волч тадолнж яервосай воуалкб тн щье воккаа яоуошто крнвхемп ксоемп швнрп янслп о доровом вейч сяевеуа а сокяаргснлкб уо йергвтнуэнралертеьо созвнкрн уомн одвпюеттгж уехесгма ьпсевтевнма внзсбзтгма то яоуошокрвнкртгма нуыщрнтрнма а явойама яолдосгма а хрнштгма лайтокрбма воуарелчтаэн еьо аз фнмалаа долбзатгц с уесаэнц н с ьетевнлчхнц ньнфодлеб дпзчматахтн давкнтосн яватнулеюнлн д йаклп мнрпхеддомнтуавх токалн ягхтге йеяэг а хпмтге хелдосге ялнрчб с эевдса яоуцоуалн яевснб до двекрп ьосовалн ьвомдо а мтоьо уояпкднлн уереж првом д впйде тн тойч ац

In [10]:
char_accuracy(tokenized_text, decoded_text)

0.4042440318302387

Результат не очень и расшифрованный текст невозможно прочитать.

**Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост:** 
*после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:*

подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;

проведите тестирование аналогично п.1, но при помощи биграмм.

In [11]:
corpus_freqs_bigram = get_freqs(tokenized_corpus, n_gram=2)
text_freqs_bigram = get_freqs(encoded_text, n_gram=2)

corpus_freqs_sorted = sorted(corpus_freqs_bigram.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs_bigram.items(), key=lambda x: x[1], reverse=True)

In [12]:
def get_reverse_mapping_bigram(corpus_freqs, text_freqs, init_mapping=None):
    corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
    text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)
    
    if init_mapping is None:
        init_mapping = []
    reverse_mapping = {k: v for k, v in init_mapping}
    
    for i, (text_bigram, text_freq) in enumerate(text_freqs_sorted):

        filtred_freqs = copy(corpus_freqs_sorted)
        
        if text_bigram[0] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[0] == reverse_mapping[text_bigram[0]]]
            
        if text_bigram[1] in reverse_mapping.keys():
            filtred_freqs = [(bigram, freq) for bigram, freq in filtred_freqs if bigram[1] == reverse_mapping[text_bigram[1]]]
              
        min_diff = 1
        best_bigram = None
        for bigram, freq in filtred_freqs:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_bigram = bigram
                min_diff = diff
                
        if text_bigram[0] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[0]] = best_bigram[0]
            
        if text_bigram[1] not in reverse_mapping.keys():
            reverse_mapping[text_bigram[1]] = best_bigram[1]

        
    return reverse_mapping

In [13]:
tokenized_text

'зовут его николаем петровичем кирсановым у него в пятнадцати верстах от постоялого дворика хорошее имение в двести душ или как он выражается с тех пор как размежевался с крестьянами и завел ферму в две тысячи десятин земли отец его боевой генерал года полуграмотный грубый но не злой русский человек всю жизнь свою тянул лямку командовал сперва бригадой потом дивизией и постоянно жил в провинции где в силу своего чина играл довольно значительную роль николай петрович родился на юге россии подобно старшему своему брату павлу о котором речь впереди и воспитывался до четырнадцатилетнего возраста дома окруженный дешевыми гувернерами развязными но подобострастными адъютантами и прочими полковыми и штабными личностями родительница его из фамилии колязиных в девицах а в генеральшах агафоклея кузьминишна кирсанова принадлежала к числу матушеккомандирш носила пышные чепцы и шумные шелковые платья в церкви подходила первая ко кресту говорила громко и много допускала детей утром к ручке на ночь их

In [14]:
encoded_text

'ашпимузюшувчьшнщзлугзмдшпчкзлуьчдрщвшпелуиувзюшупугэмвщыъщмчупздрмщхушмугшрмшэншюшуыпшдчьщухшдшфззучлзвчзупуыпзрмчуыифучнчуьщьушвупедщжщзмрэурумзхугшдуьщьудщалзжзпщнрэуруьдзрм эвщлчучуащпзнуйздлиупуыпзумерэкчуызрэмчвуазлнчушмзъузюшусшзпшбуюзвздщнуюшыщугшниюдщлшмвебуюдисебувшувзуаншбудиррьчбукзншпзьупроужчав урпшоумэвинунэльиуьшлщвышпщнургздпщусдчющышбугшмшлуычпчачзбучугшрмшэввшужчнупугдшпчвъччуюызупурчниурпшзюшукчвщучюдщнуышпшн вшуавщкчмзн виоудшн увчьшнщбугзмдшпчкудшычнрэувщуоюзудшррччугшышсвшурмщдфзлиурпшзлиусдщмиугщпниушуьшмшдшлудзк упгздзычучупшргчмепщнрэуышукзмедвщыъщмчнзмвзюшупшадщрмщуышлщушьдижзввебуызфзпелчуюипздвздщлчудщапэавелчувшугшышсшрмдщрмвелчущыцомщвмщлчучугдшкчлчугшньшпелчучуфмщсвелчунчквшрмэлчудшычмзн вчъщузюшучауйщлчнччуьшнэачвехупуызпчъщхущупуюзвздщн фщхущющйшьнзэуьиа лчвчфвщуьчдрщвшпщугдчвщынзжщнщуьукчрниулщмифзььшлщвычдфувшрчнщугефвезукзгъеучуфилвезуфзньшпезугнщм эупуъздьпчугшыхшычнщугздпщэуьшуьдзрмиуюшпшдчнщуюдшльшучулвшюшуышгирьщнщуызмзбуимдшлуьудикьзувщувшк учх

In [15]:
reverse_mapping_bigram = get_reverse_mapping_bigram(corpus_freqs_bigram, text_freqs_bigram)
decoded_text = apply_mapping(encoded_text, reverse_mapping_bigram)
decoded_text

'баньо яна иоиалеял сяонаносял ион еианлл ь ияна н своиеичеоо нян оею ао са оавлана инаноие юанасяя оляиоя н иня оо иьс оло иеи аи нлнемеяо в   ояю сан иеи неблямянел в   иня оовиело о бенял шянль н иня ол всо ия воои бялло аояч яна еаяная няиянел наие сальннелаоиля нньеля иа ия блая нь  иоя сяланяи н с мобио  нас овиьл лвлиь иалеиианел  сянне енонеиая саоал ионобояя о са оавииа мол н снаноичоо ния н  оль  наяна соие оннел ианалоиа биесооялоиьс нало иоиалея сяонанос наиол в ие сня на  оо саиаеиа  оенсяль  наяль енеоь сенль а иаоанал нясо нсяняио о на соолнел в иа сяолниеичеооляоияна набне ое иале аиньмяииля иясянлло ньняниянело небнвбилло иа саиаеа оне оилло еирсоеиоело о снасоло салианлло о соееилло лосиа овло наиооялоиоче яна об шелолоо иалвбоилю н ияночею е н няиянелосею енешаиляв иьболоиосие ион еиане сноиеилямеле и со ль леоьсяииалеиионс иа оле слсиля сясчл о сьлиля сялианля слеоов н чянино саиюаиоле сяннев иа иня оь нананоле нналиа о лиана иась иеле ияояя ьонал и ньсия ие иасо ою

In [16]:
char_accuracy(tokenized_text, decoded_text)

0.1962864721485411

Результат хуже. Наверно нужен очень большой текст...

**Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:**

предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;

реализуйте и протестируйте его, убедитесь, что результаты улучшились.


У нас есть биграммы. Представим что у нас Марковская цепь. Частота биграмм есть вероятность перехода по цепи. 
Алгоритм:
1. Инициализируем перестановки, восстанавливаем текст и считаем на нем текующую вероятность.
2. Меняем местами пару букв
3. Восстанавливаем текст с новой перестановкиой и считаем на нём вероятность
4. Принимаем новую перестановку с вероятностью равной отношеню веротяности новой перестановки на текущую


In [23]:
def get_freqs_smooth(text, min_freq=0, n_gram=2):
    freqs = dict()
    vocab_len = len(set(text))**n_gram
    if n_gram > 1:
        text = [''.join(ngram) for ngram in everygrams(text, min_len=n_gram, max_len=n_gram)]
    for key, value in Counter(text).items():
        freqs[key] = (value + 1) / (len(text) + vocab_len) # Сглаживаем, чтобы не было нулей
    return freqs

def get_text_proba(text, mapping, freqs, n_gram=2):
    decoded_text = apply_mapping(text, mapping)
    log_proba = 0
    for i in range(len(decoded_text) - n_gram):
        bigram = decoded_text[i: i + n_gram]
        bigram_proba = freqs.get(bigram)
        if bigram_proba is None:
            bigram_proba = 1 / (len(text) + len(alphabet)**n_gram) # Сглаживаем, чтобы не было нулей
            
        log_proba += np.log(bigram_proba)
    return log_proba
        
def get_reverse_mapping_mcmc(encoded_text, alphabet_encoded, alphabet_corpus, freqs_corpus, n_iters=10000, n_trials=10, n_gram=2):

    accept_count = 0
    best_mapping = None
    all_mappings = []
    best_log_likekihood = -np.inf

    for trial in tqdm(range(n_trials), leave=False, position=0, total=n_trials):

        alphabet_encoded = list(alphabet_encoded)
        alphabet_iter = list(alphabet_corpus)
        reverse_mapping = {k: v for k, v in zip(alphabet_encoded, alphabet_iter[:len(alphabet_encoded)])}
        log_proba_current = get_text_proba(encoded_text, reverse_mapping, freqs_corpus, n_gram=n_gram)

        for i in range(n_iters):
            alphabet_proposal = alphabet_iter[:]
            idx1, idx2 = np.random.choice(len(alphabet_proposal), replace=False, size=2)
            alphabet_proposal[idx1], alphabet_proposal[idx2] = alphabet_proposal[idx2], alphabet_proposal[idx1]
            reverse_mapping_proposal = {k: v for k, v in zip(alphabet_encoded, alphabet_proposal[:len(alphabet_encoded)])}
            log_proba_proposal = get_text_proba(encoded_text, reverse_mapping_proposal, freqs_corpus, n_gram=n_gram)

            p_accept = np.exp(log_proba_proposal - log_proba_current)

            if p_accept > np.random.rand():
                accept_count += 1
                alphabet_iter = alphabet_proposal
                log_proba_current = log_proba_proposal
                reverse_mapping = reverse_mapping_proposal

        if log_proba_current > best_log_likekihood:
            best_log_likekihood = log_proba_current
            best_mapping = reverse_mapping
            
        all_mappings.append(reverse_mapping)


    print(f'Best likelihood: {best_log_likekihood}')        
    print(f'Accept raito: {accept_count / (n_iters * n_trials)}')
    return best_mapping, all_mappings

In [24]:
freqs_corpus = get_freqs_smooth(tokenized_corpus, n_gram=2)

In [25]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    alphabet_encoded=alphabet, 
    alphabet_corpus=alphabet,
    freqs_corpus=freqs_corpus
)

Best likelihood: -10611.230112139352
Accept raito: 0.01236


In [26]:
apply_mapping(encoded_text, best_reverse_mapping)

'зовут его николаем петровичем кирсановым у него в пятнадцати верстах от постоялого дворика хорошее имение в двести душ или как он выражается с тех пор как размежевался с крестьянами и завел эерму в две тысячи десятин земли отец его боевой генерал года полуграмотный грубый но не злой русский человек всю жизнь свою тянул лямку командовал сперва бригадой потом дивизией и постоянно жил в провинции где в силу своего чина играл довольно значительную роль николай петрович родился на юге россии подобно старшему своему брату павлу о котором речь впереди и воспитывался до четырнадцатилетнего возраста дома окруженный дешевыми гувернерами развязными но подобострастными адъютантами и прочими полковыми и штабными личностями родительница его из эамилии колязиных в девицах а в генеральшах агаэоклея кузьминишна кирсанова принадлежала к числу матушеккомандирш носила пышные чепцы и шумные шелковые платья в церкви подходила первая ко кресту говорила громко и много допускала детей утром к ручке на ночь их

**Расшифруйте сообщение:**

In [48]:
msg = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [49]:
corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
message_freqs_sorted = sorted(Counter(msg).items(), key=lambda x: x[1], reverse=True)

alphabet_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
alphabet_message = ''.join([c for c, _ in message_freqs_sorted])
alphabet_corpus, abc_message

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', '↹←⇛↟⇒↝⇴↨⇠⇯↷⇌⇊⇞⇈⇷↤↳↾↙⇙↲↞⇆⇰⇸↘⇏')

In [50]:
freqs_corpus = get_freqs_smooth(tokenized_corpus, n_gram=2)

In [51]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    message, 
    alphabet_encoded=alphabet_message, 
    alphabet_corpus=alphabet_corpus,
    freqs_corpus=freqs_corpus,
    n_iters=10000,
    n_trials=50,
)

Best likelihood: -1231.2547551943435
Accept raito: 0.042772


In [52]:
apply_mapping(msg, best_reverse_mapping)

'если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаю'

**Бонус:** 

а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [53]:
corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
message_freqs_sorted = sorted(Counter(msg).items(), key=lambda x: x[1], reverse=True)

alphabet_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
alphabet_message = ''.join([c for c, _ in message_freqs_sorted])
alphabet_corpus, alphabet_message

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', '↹←⇛↟⇒↝⇴↨⇠⇯↷⇌⇊⇞⇈⇷↤↳↾↙⇙↲↞⇆⇰⇸↘⇏')

In [54]:
freqs_corpus_trigramm = get_freqs_smooth(tokenized_corpus, n_gram=3)

In [55]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    message, 
    alphabet_encoded=alphabet_message, 
    alphabet_corpus=alphabet_corpus,
    freqs_corpus=freqs_corpus_trigramm,
    n_gram=3,
    n_iters=10000,
    n_trials=20,
)

Best likelihood: -1725.6602016838426
Accept raito: 0.042765


In [56]:
apply_mapping(msg, best_reverse_mapping)

'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаъ'

Попробуем длинный текст

In [77]:
mapping = generate_mapping(corpus_freqs)
encoded_text = apply_mapping(tokenized_text, mapping)
text_freqs = get_freqs(encoded_text)

corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

alphabet_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
alphabet_text = ''.join([c for c, _ in text_freqs_sorted])
alphabet_corpus, alphabet_text

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', 'зеипчорщфйбмуцышю дьтлсакхявжгнъ')

In [78]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    alphabet_encoded=alphabet_text, 
    alphabet_corpus=alphabet_corpus,
    freqs_corpus=freqs_corpus,
    n_gram=2,
    n_iters=10000,
    n_trials=5,
)
print(encoded_text)
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print(char_accuracy(tokenized_text, decoded_text))

Best likelihood: -10611.230112139352
Accept raito: 0.00816
терыйзиюезопмещчиузшийферпсиузмпфбчоер узызоиюезрзшдйочцячйпзрифбйчвзейзшебйедщеюезцрефпмчзвефекиизпуиопизрзцрибйпзцыкзпщпзмчмзеозр фчхчийбдзбзйивзшефзмчмзфчтуихирчщбдзбзмфибйьдочупзпзтчрищзгифуызрзцризй бдспзцибдйпозтиущпзейиязиюезаеирелзюиоифчщзюецчзшещыюфчуейо лзюфыа лзоезоизтщелзфыббмплзсищеримзрбжзхптоьзбрежзйдоыщзщдумызмеучоцерчщзбшифрчзафпючцелзшейеузцпрптпилзпзшебйедооезхпщзрзшферпояппзюцизрзбпщызбреиюезспочзпюфчщзцерещьоезточспйищьоыжзфещьзопмещчлзшийферпсзфецпщбдзочзжюизфеббппзшецеаоезбйчфкиуызбреиуызафчйызшчрщызезмейефеузфисьзршифицпзпзребшпй рчщбдзцезсий фочцячйпщийоиюезретфчбйчзцеучземфыхиоо лзцикир упзюырифоифчупзфчтрдто упзоезшецеаебйфчбйо упзчцъжйчойчупзпзшфеспупзшещмер упзпзкйчао упзщпсоебйдупзфецпйищьопячзиюезптзгчупщппзмещдтпо взрзцирпячвзчзрзюиоифчщькчвзчючгемщидзмытьупопкочзмпфбчоерчзшфпочцщихчщчзмзспбщызучйыкиммеучоцпфкзоебпщчзш ко изсишя зпзкыуо изкищмер изшщчйьдзрзяифмрпзшецвецпщчзшифрчдзмезмфибйызюерефп

С относительно большим текстом биграммы показали отличный результат

In [81]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    alphabet_encoded=alphabet_text, 
    alphabet_corpus=alphabet_corpus,
    freqs_corpus=freqs_corpus_trigramm,
    n_gram=3,
    n_iters=10000,
    n_trials=5,
)

print(encoded_text)
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print(char_accuracy(tokenized_text, decoded_text))

 20%|██        | 1/5 [00:34<02:16, 34.14s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: overflow encountered in exp
                                             

Best likelihood: -14811.27751124043
Accept raito: 0.00664
терыйзиюезопмещчиузшийферпсиузмпфбчоер узызоиюезрзшдйочцячйпзрифбйчвзейзшебйедщеюезцрефпмчзвефекиизпуиопизрзцрибйпзцыкзпщпзмчмзеозр фчхчийбдзбзйивзшефзмчмзфчтуихирчщбдзбзмфибйьдочупзпзтчрищзгифуызрзцризй бдспзцибдйпозтиущпзейиязиюезаеирелзюиоифчщзюецчзшещыюфчуейо лзюфыа лзоезоизтщелзфыббмплзсищеримзрбжзхптоьзбрежзйдоыщзщдумызмеучоцерчщзбшифрчзафпючцелзшейеузцпрптпилзпзшебйедооезхпщзрзшферпояппзюцизрзбпщызбреиюезспочзпюфчщзцерещьоезточспйищьоыжзфещьзопмещчлзшийферпсзфецпщбдзочзжюизфеббппзшецеаоезбйчфкиуызбреиуызафчйызшчрщызезмейефеузфисьзршифицпзпзребшпй рчщбдзцезсий фочцячйпщийоиюезретфчбйчзцеучземфыхиоо лзцикир упзюырифоифчупзфчтрдто упзоезшецеаебйфчбйо упзчцъжйчойчупзпзшфеспупзшещмер упзпзкйчао упзщпсоебйдупзфецпйищьопячзиюезптзгчупщппзмещдтпо взрзцирпячвзчзрзюиоифчщькчвзчючгемщидзмытьупопкочзмпфбчоерчзшфпочцщихчщчзмзспбщызучйыкиммеучоцпфкзоебпщчзш ко изсишя зпзкыуо изкищмер изшщчйьдзрзяифмрпзшецвецпщчзшифрчдзмезмфибйызюерефпщ

In [82]:
freqs_corpus_fourgramm = get_freqs_smooth(tokenized_corpus, n_gram=4)

In [83]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    alphabet_encoded=alphabet_text, 
    alphabet_corpus=alphabet_corpus,
    freqs_corpus=freqs_corpus_fourgramm,
    n_gram=4,
    n_iters=10000,
    n_trials=5,
)

print(encoded_text)
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print(char_accuracy(tokenized_text, decoded_text))

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: overflow encountered in exp
                                             

Best likelihood: -18914.225859776645
Accept raito: 0.00482
терыйзиюезопмещчиузшийферпсиузмпфбчоер узызоиюезрзшдйочцячйпзрифбйчвзейзшебйедщеюезцрефпмчзвефекиизпуиопизрзцрибйпзцыкзпщпзмчмзеозр фчхчийбдзбзйивзшефзмчмзфчтуихирчщбдзбзмфибйьдочупзпзтчрищзгифуызрзцризй бдспзцибдйпозтиущпзейиязиюезаеирелзюиоифчщзюецчзшещыюфчуейо лзюфыа лзоезоизтщелзфыббмплзсищеримзрбжзхптоьзбрежзйдоыщзщдумызмеучоцерчщзбшифрчзафпючцелзшейеузцпрптпилзпзшебйедооезхпщзрзшферпояппзюцизрзбпщызбреиюезспочзпюфчщзцерещьоезточспйищьоыжзфещьзопмещчлзшийферпсзфецпщбдзочзжюизфеббппзшецеаоезбйчфкиуызбреиуызафчйызшчрщызезмейефеузфисьзршифицпзпзребшпй рчщбдзцезсий фочцячйпщийоиюезретфчбйчзцеучземфыхиоо лзцикир упзюырифоифчупзфчтрдто упзоезшецеаебйфчбйо упзчцъжйчойчупзпзшфеспупзшещмер упзпзкйчао упзщпсоебйдупзфецпйищьопячзиюезптзгчупщппзмещдтпо взрзцирпячвзчзрзюиоифчщькчвзчючгемщидзмытьупопкочзмпфбчоерчзшфпочцщихчщчзмзспбщызучйыкиммеучоцпфкзоебпщчзш ко изсишя зпзкыуо изкищмер изшщчйьдзрзяифмрпзшецвецпщчзшифрчдзмезмфибйызюерефп

In [84]:
short_text = 'Николай Петрович быстро обернулся и, подойдя к человеку высокого роста в длинном балахоне с кистями, только что вылезшему из тарантаса, крепко стиснул его обнаженную красную руку, которую тот не сразу ему подал'

In [85]:
mapping = generate_mapping(corpus_freqs)
encoded_text = apply_mapping(short_text, mapping)
text_freqs = get_freqs(encoded_text)

corpus_freqs_sorted = sorted(corpus_freqs.items(), key=lambda x: x[1], reverse=True)
text_freqs_sorted = sorted(text_freqs.items(), key=lambda x: x[1], reverse=True)

alphabet_corpus = ''.join([c for c, _ in corpus_freqs_sorted])
alphabet_text = ''.join([c for c, _ in text_freqs_sorted])
alphabet_corpus, alphabet_text

(' оеанитслвркдмупяьгыбзчжйшхюэцщфъ', 'яйьпу ктмвыжсеюдиагъзэфнлщхц')

In [86]:
freqs_corpus_fourgramm = get_freqs_smooth(tokenized_corpus, n_gram=4)

In [88]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    alphabet_encoded=alphabet_text, 
    alphabet_corpus=alphabet_corpus,
    freqs_corpus=freqs_corpus_fourgramm,
    n_gram=4,
    n_iters=10000,
    n_trials=5,
)

print(encoded_text)
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print(char_accuracy(short_text, decoded_text))

Best likelihood: -2102.0483719206836
Accept raito: 0.03346
ьж йыьфяьпувйсжияеакувйяйепвтмыкгяжьяъйюйфюгя яипыйсп мясакй йнйявйкуьясяюыжттйдяеьыьлйтпякя жкугджьяуйыщ йяиуйясаыпзхпдмяжзяуьвьтуькьья впъ йякужктмыяпнйяйетьцпттмэя вьктмэявм мья йуйвмэяуйуятпяквьзмяпдмяъйюьы
аиколай аетрович быстро обернулся иа подойдя к человеку высокого роста в длинном балахоне с кистямиа только что вылезшему из тарантасаа крепко стиснул его обнаженную красную рукуа которую тот не сразу ему подал
0.9714285714285714


In [89]:
freqs_corpus_trigramm = get_freqs_smooth(tokenized_corpus, n_gram=3)

In [90]:
best_reverse_mapping, _ = get_reverse_mapping_mcmc(
    encoded_text, 
    alphabet_encoded=alphabet_text, 
    alphabet_corpus=alphabet_corpus,
    freqs_corpus=freqs_corpus_trigramm,
    n_gram=3,
    n_iters=10000,
    n_trials=5,
)

print(encoded_text)
decoded_text = apply_mapping(encoded_text, best_reverse_mapping)
print(decoded_text)
print(char_accuracy(short_text, decoded_text))

Best likelihood: -1675.6677374485785
Accept raito: 0.0377
ьж йыьфяьпувйсжияеакувйяйепвтмыкгяжьяъйюйфюгя яипыйсп мясакй йнйявйкуьясяюыжттйдяеьыьлйтпякя жкугджьяуйыщ йяиуйясаыпзхпдмяжзяуьвьтуькьья впъ йякужктмыяпнйяйетьцпттмэя вьктмэявм мья йуйвмэяуйуятпяквьзмяпдмяъйюьы
аьколах аетровьч быстро обернился ьа подохдя к человеки высокого роста в дльнном балаъоне с кьстямьа толуко что вылезщеми ьз тарантасаа крепко стьснил его обнашенний красний рикиа которий тот не срази еми подал
0.8380952380952381


Вывод: При достаточно большом тексте 4-граммы работают отлично. Возможно при правильном подборе гиперапаметров и увеличении размера n-грамм точность тоже увеличивается даже для коротких текстов

**Бонус: какие вы можете придумать применения для этой модели?**
 Пляшущие человечки ведь не так часто встречаются в жизни 
 (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).



Возможно что такие модели можно использовать для сжатия больших данных и передачи информации.
Также возможно применение в биоинформатике для работы с последовательностями ДНК и РНК